In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('./data/sp500_stocks.csv')

df.shape

(1890274, 8)

In [3]:
n_rows_na= df.isna().any(axis=1).sum()
print(f"Number of rows with atleast one Nan value: {n_rows_na}")

Number of rows with atleast one Nan value: 101626
